In [ ]:
import sys
sys.path.append("../dataprep")
from dglUtils import *

In [ ]:
f1='../../../../Desktop/CERNBox/LRJgraph.csv'
f2='../../../../Desktop/CERNBox3/LRJgraph.csv'
p=merge_datasets(f1,f2,isGlobal=True)
p.to_csv( "../dataset/LRJgraph.csv", index=False, encoding='utf-8-sig')

In [ ]:
f1='../../../../Desktop/CERNBox/tracknodes.csv'
f2='../../../../Desktop/CERNBox3/tracknodes.csv'
p=merge_datasets(f1,f2,isGlobal=False)
p.to_csv( "../dataset/tracknodes.csv", index=False, encoding='utf-8-sig')

In [ ]:
f1='../../../../Desktop/CERNBox/calonodes.csv'
f2='../../../../Desktop/CERNBox3/calonodes.csv'
p=merge_datasets(f1,f2,isGlobal=False)
p.to_csv( "../dataset/calonodes.csv", index=False, encoding='utf-8-sig')

In [ ]:
f1='../../../../Desktop/CERNBox/subjets.csv'
f2='../../../../Desktop/CERNBox3/subjets.csv'
p=merge_datasets(f1,f2,isGlobal=False)
p.to_csv( "../dataset/subjetnodes.csv", index=False, encoding='utf-8-sig')

In [ ]:
#Create dask service
cluster = LocalCluster()
client = Client(cluster)

In [ ]:
#Create dask dataframes to hold the input csv files
cell_df = dd.read_csv('../dataset/calonodes.csv',dtype={'graph_id': np.int64})
track_df = dd.read_csv('../dataset/tracknodes.csv',dtype={'graph_id': np.int64})
subjet_df = dd.read_csv('../dataset/subjetnodes.csv',dtype={'graph_id': np.int64})
global_df = dd.read_csv('../dataset/LRJgraph.csv',dtype={'graph_id': np.int64})
df_dict = {'cells':cell_df,'tracks':track_df,'subjets':subjet_df,'global':global_df}

In [ ]:
#Create node num dictionary for graph creation
n_sizedict={}
for key in df_dict.keys():
    n_sizedict[key]=get_nNodes(df_dict[key])

In [ ]:
#Create node feature value dictionary for graph creation
node_type_feats = {'cells': ['node_id', 'center', 'E', 'type'],
    'tracks': ['node_id', 'center', 'E', 'D0'],
    'subjets': ['node_id', 'center', 'E', 'TruthMatch'],
    'global': ['node_id', 'center', 'E', 'TruthMatch']}
n_featdict={}
for key in node_type_feats.keys():
    n_featdict[key]=create_node_featdict(df_dict[key],node_type_feats,ntype=key)

In [ ]:
#Create the graph list (must specify at least one edge type)
graphs = []
data_dict = {
    ('cells', 'cells2cells', 'cells'): ([], []),
    ('tracks', 'tracks2tracks', 'tracks'): ([], []),
            }
num_graphs = len(n_sizedict['global'])
for i in tqdm(range(num_graphs)):
    graphs.append(create_graph(i,n_featdict=n_featdict,n_sizedict=n_sizedict,e_dict=data_dict))

In [ ]:
g_balanced = get_TMBalanced_dataset(graphs,randseed=234)
len(g_balanced)

In [ ]:
save_graphs("../dataset/DGLdataset_balanced.bin",g_balanced)